In [1]:
import numpy as np
import pandas as pan
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt

import os
from datetime import datetime

In [2]:
df = pan.DataFrame(columns=['timestamp', 'source', 'destination', 'bandwidth'])
ini = False
progress = 0
start = 10000
endFile = 10772
numberOfFiles = 10772
numberOfParsedFiles = 0
dataframes = []
for root, dirs, files in os.walk(r'..\Datasets\traffic-matrices-anonymized-v2\traffic-matrices'):
    for file in sorted(files):
        path = root + '\\' + file
        if str(path).endswith('.xml'):
            if progress >= start and progress < endFile:
                dataframes.append(pan.DataFrame(columns=['timestamp', 'source', 'destination', 'bandwidth']))
                #Parse xml file
                tree = ET.parse(path)
                treeRoot = tree.getroot()
                #parse date
                info = treeRoot.find("info")
                date = info.find("date")
                dateObject = pan.to_datetime(date.text, format='%Y-%m-%dT%H:%M:%S')
                #parse records
                intraTm = treeRoot.find("IntraTM")
                srcList = intraTm.findall("src")
                for src in srcList:
                    dstList = src.findall("dst")
                    for dst in dstList:
                        new_row = {'timestamp':dateObject, 'source':src.get('id'), 'destination':dst.get('id'), 'bandwidth':dst.text}
                        dataframes[numberOfParsedFiles] = dataframes[numberOfParsedFiles].append(new_row, ignore_index=True)                        
                numberOfParsedFiles += 1
                
            progress += 1
            x = (100/endFile) * progress
            print('\r{0}%                 '.format(x), end='\r')
        if numberOfParsedFiles == endFile:
            break
    if numberOfParsedFiles == endFile:
        break
df = pan.concat(dataframes)

64.82547344968437%                 

In [3]:
path = r'..\Datasets\GEANTCombined\all_in_one_part11.csv'
df.to_csv(path, encoding='utf-8', index=False)

In [4]:
#df.plot(y = ["bandwidth"], figsize=(20,10)).set(xlabel="Timestamp", ylabel="Bandwidth")